In [1]:
import os
import pyvips
import cv2
import subprocess
import numpy as np
from osgeo import gdal
from PIL import Image

In [4]:
Image.MAX_IMAGE_PIXELS = 9966240000

#### (older) Breaking the PNG into smaller tiles using PIL

In [ ]:
# def create_tile(filename, dir_in, dir_out, d, rotation):
#     name, ext = os.path.splitext(filename)
#     img = Image.open(os.path.join(dir_in, filename))
#     w, h = img.size

#     grid = product(range(0, h-h%d, d), range(0, w-w%d, d))
#     for i, j in grid:
#         box = (j, i, j+d, i+d)
#         out = os.path.join(dir_out, f'tile{rotation}_{i}_{j}{ext}')
#         img.crop(box).save(out)

In [ ]:
# filename = 'SRTM_Scaled_UInt16_Europe.png'
# dir_in = './SRTM_Europe/'
# dir_out = './Tiles/Europe/'
# d = 1024
# rotation = 0
# create_tile(filename, dir_in, dir_out, d, rotation)

#### Fill transparent areas in raster

In [ ]:
# def fillWhite(filename, dir_in, dir_out):
#     image = Image.open(os.path.join(dir_in, filename))
#     image.convert("RGBA")
    
#     canvas = Image.new('RGBA', image.size, (65535, 65535, 65535, 65535))
#     canvas.paste(image, mask=image)
#     canvas.convert("L")
#     canvas.save(f'{os.path.join(dir_out, name)}_filled.png')

#### converting the colourspace in pyvips

In [ ]:
# npimg = np.ndarray(buffer = img.write_to_memory(),
    #                   dtype = np.uint16,
    #                   shape = [img.height, img.width, 1])
    
    
    # img2 = pyvips.Image.new_from_array(npimg)
    
    # img2 = img.bandsplit()[1]
    
    # print(img2.bands)
    # print(img2.format)
    # print(img2.width, img2.height)

    # img2 = img.colourspace('b-w')
    
    # mem_img = img.write_to_memory()
    # imgnp = np.frombuffer(mem_img, dtype=np.uint16).reshape(img.width, img.height, 1)
    

## Rotate the raster using pyvips

In [3]:
def rotateRaster(filename, dir_in, dir_out, rotation):
    
    name, ext = os.path.splitext(filename)
    
    # Open the image using Pyvips
    img = pyvips.Image.pngload(os.path.join(dir_in, filename))
    print(f'Rotating "{filename}" by {rotation} degrees...')
    # Rotate the image without specifying interpolation (defaults to nearest neighbor)
    
    if rotation == 90:
        rot = img.rot90()
    else:
        rot = img.rotate(rotation)
    
    op_filename = f'{name} {rotation}.png'
    op_file = f'{os.path.join(dir_out, op_filename)}'
    
    rot.pngsave(op_file)
    
    img= None
    rot = None
    print(f'"{filename}" was successfully rotated by {rotation} degrees.')
    
    return op_filename


pngsave does work, but adds an alpha layer which needs to be removed using gdal_translate

## Flip the raster Horizontally

In [4]:
def flipRaster(filename, dir_in, dir_out):
    
    name, ext = os.path.splitext(filename)
    
    # Open the image using Pyvips
    img = pyvips.Image.pngload(os.path.join(dir_in, filename))
    print(f'Flipping "{filename}"...')

    rot = img.fliphor()
    
    op_filename = f'{name} flipped.png'
    op_file = f'{os.path.join(dir_out, op_filename)}'
    
    rot.pngsave(op_file)
    
    img = None
    rot = None
    print(f'"{filename}" was successfully horizontally flipped.')
    
    return op_filename

## Extract only the Gray Band

In [5]:
def extractBand(filename, dir_in, dir_out_rot, rotation):
    name, ext = os.path.splitext(filename)
    name, ang = name.split(' ')
    
    ip_path = f'{os.path.join(dir_in, filename)}'
    op_filename = f'{name}_{rotation}.png'
    op_path = f'{os.path.join(dir_in, op_filename)}'
        
    ds = gdal.Open(ip_path)

    subprocess.run(['gdal_translate', '-b', '1', ip_path, op_path])
    
    ds = None
    print(f'Alpha band was removed from "{filename}".')
    return op_filename
    

## Breaking the PNG into smaller tiles using PIL

In [6]:
def createTiles(filename, dir_in, dir_out, rotation, region):
    print(f'Creating Tiles from "{filename}"...')
    name, ext = os.path.splitext(filename)
    tname = f'tile{region}{rotation}'
    rotated_image = Image.open(os.path.join(dir_in, filename))
    
    tile_size = 512
    window_size = 256
    f_count = 0
    # Assign pixel_count and threshold here since it'll be the same for all tiles
    pixel_count = 262144 # tile.width * tile.height, for tiles of 1024x1024: 1024*1024 = 1048576
    threshold = 249037 # int(0.95 * pixel_count), for tiles of 1024x1024: 0.95 * 1048576 = 996147
    
    for y in range(0, rotated_image.height - tile_size + 1, window_size):
        for x in range(0, rotated_image.width - tile_size + 1, window_size):
            
            tile = rotated_image.crop((x, y, x + tile_size, y + tile_size))
            
            count = 0
            
            if max(tile.getextrema()) == 255:
                continue
            
            pixels = tile.load()
            for i in range(tile.size[0]):  # Loop through each pixel in the tile
                for j in range(tile.size[1]):
                    if pixels[i, j] <= 25:
                        count += 1
            pixels = None
            
            if count < threshold:
                f_count += 1
                tile.save(f'{os.path.join(dir_out, tname)}_{x}_{y}.png')
            tile = None
    
    rotated_image = None            
    print(f'{f_count} Tiles were generated from {filename}.')
            

#### Function to check im.getdata() results

In [7]:
# def check(filename, dir_in):
#     img = Image.open(os.path.join(dir_in, filename))
#     tile = img.crop((0, 0, 0 + 1024, 0 + 1024))
    
#     pixels_count = tile.width*tile.height
#     threshold = int(0.95 * pixels_count)
    
#     pixels = tile.getdata()
    
#     print("Threshold = ", threshold)
#     print("len(tile.getdata()) = ", len(pixels))
#     print("1024 * 1024 = ", pixels_count)

# check("SRTM_Europe_UInt16_Scaled.png", './SRTM_Europe/scaled/')

## Caller Function

In [7]:
def caller():   
    
    f_name = {
        'E8' : 'SRTM_UInt16_to_Byte_Europe.png',
        'I8' : 'SRTM_UInt16_to_Byte_India.png'
    }
    
    ip_path = {
        'E8' : '/base/Projects/TerrainGen/SRTM_Europe/non-scaled/',
        'I8' : '/base/Projects/TerrainGen/SRTM_India/non-scaled/'
    }
    
    op_path = {
        'E8' : '/base/Projects/TerrainGen/Tiles/generated_tiles/',
        'I8' : '/base/Projects/TerrainGen/Tiles/generated_tiles/'
    }
    
    regions = ["E8"] # I8
    for region in regions:
    
        angles = [33] # 0, 'f'
        for deg in angles:
            
            filename = f_name[region]
            dir_in = ip_path[region]
            dir_out_rot = ip_path[region]
            dir_out = op_path[region]
            
            # functions calls
            
            if deg == 0:
                # since the tiles will be generated from the input raster itself, extractBand method need not be called.
                singleband_fname = filename
            
            elif deg == 'f':
                fname = flipRaster(filename, dir_in, dir_out_rot)
                singleband_fname = extractBand(fname, dir_in, dir_out_rot, deg)
            
            else:
                fname = rotateRaster(filename, dir_in, dir_out_rot, deg)
                singleband_fname = extractBand(fname, dir_in, dir_out_rot, deg)
            
            createTiles(singleband_fname, dir_in, dir_out, deg, region)

In [8]:
caller()

Rotating "SRTM_UInt16_to_Byte_Europe.png" by 33 degrees...
"SRTM_UInt16_to_Byte_Europe.png" was successfully rotated by 33 degrees.
Input file size is 76072, 63370
0...10...20...30...40...50...60...70...80...90...100 - done.
Alpha band was removed from "SRTM_UInt16_to_Byte_Europe 33.png".
Creating Tiles from "SRTM_UInt16_to_Byte_Europe_33.png"...
5714 Tiles were generated from SRTM_UInt16_to_Byte_Europe_33.png.


## Count files

In [9]:
count = 0
out_path = "./Tiles/Train/"

# Iterate directory
for path in os.listdir(out_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(out_path, path)):
        count += 1

print('File count in generated_tiles: ', count)

File count in generated_tiles:  5696


## Downscale the dataset to 128x128

In [7]:
def downscale(dir_in, dir_out, no_of_tiles):
    files = os.listdir(dir_in)
    print(f'downscaling {no_of_tiles} images...')
    count = 0
    for file_name in files:
        if count == no_of_tiles:
            break
        
        if file_name.endswith(".png"):
            
            input_path = os.path.join(dir_in, file_name)
            output_path = os.path.join(dir_out, file_name)

            # Open the image
            image = Image.open(input_path)

            # Resize the image using nearest neighbor interpolation
            resized_image = image.resize((128, 128), resample=Image.NEAREST)

            # Save the image in the output directory
            resized_image.save(output_path)
            count += 1

In [8]:
in_path = './Tiles/generated_tiles/'
out_path = './Tiles/Train/'

downscale(in_path, out_path, 5696)

downscaling 5696 images...
